# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118007e+02     1.737109e+00
 * time: 1.1061418056488037
     1     1.090166e+01     9.940736e-01
 * time: 3.7174339294433594
     2    -1.150064e+01     1.124580e+00
 * time: 4.364718914031982
     3    -3.402755e+01     8.618323e-01
 * time: 5.3089759349823
     4    -4.756710e+01     5.494732e-01
 * time: 6.478245973587036
     5    -5.680619e+01     2.585879e-01
 * time: 7.4682958126068115
     6    -5.966406e+01     1.841496e-01
 * time: 8.090849876403809
     7    -6.085259e+01     7.800898e-02
 * time: 8.681526899337769
     8    -6.135580e+01     6.667444e-02
 * time: 9.283129930496216
     9    -6.162752e+01     6.427943e-02
 * time: 9.906085014343262
    10    -6.184125e+01     3.419281e-02
 * time: 10.48379898071289
    11    -6.199207e+01     2.348067e-02
 * time: 11.104430913925171
    12    -6.206690e+01     1.622964e-02
 * time: 11.755483865737915
    13    -6.211174e+01     1.849791e-02
 * time: 12.387220859527588
   

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671066
    AtomicLocal         -18.8557679
    AtomicNonlocal      14.8522649
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485370 
    Xc                  -19.3336819

    total               -62.261666462216
